#Summary

The jupyter notebook is used to help write the python file in `utilis` floder

For single fast5 feature extraction, see daily/2021-10-05/feature_100521.ipynb

In [ ]:
%reset

In [ ]:
import pandas as pd
import numpy as np
import sys, os
import h5py
import logging
import statsmodels
from datetime import datetime
from statsmodels import robust

from utils import process_helper, fast5_parser

## Extract signal, align info, event from Single fast5 file

In [197]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
fast5_fn='00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5'

signal_group='Raw/Reads'
corr_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl

In [198]:
path = os.path.join(input_path, fast5_fn)
fast5 = fast5_parser.raw_fast5(path, corr_group, basecall_group, basecall_subgroup, signal_group)
readid, fast5_signal = fast5.fast5_signal()
event = fast5.fast5_event()
chrom, strand, start, read_strand = fast5.fast5_align()

# Rescale signal
Copy from feature_100521.ipynb

## Define signal rescalling

In [42]:
# Signal rescalling
#current in pA = (signal_value + channels 0pA adc) * digitisable range in pA / digitisation
#https://github.com/grimbough/IONiseR/blob/47d8ab1e1d798f3591407be679076a1a5b5d9dd2/R/squiggle.R#L81

#channels 0pA adc: /UniqueGlobalKey/channel_id/{offset}
#digitisable range in pA: /UniqueGlobalKey/channel_id/{range}
#digitisation: /UniqueGlobalKey/channel_id/{digitisation}

def fast5_rescale(path, fast5_signal, channel_group='UniqueGlobalKey/channel_id'):
    try:
        f5 = h5py.File(path, mode="r")
    except IOError:
        raise IOError('Error opening file')
    
    channel = f5[f'/{channel_group}']
    channel_attr = dict(list(channel.attrs.items()))
    raw_signal = np.array((fast5_signal + channel_attr['offset']) * channel_attr['range'] / channel_attr['digitisation']) 
    return raw_signal

# Signal normalization with mad method
def fast5_normalized_signal(signal, method='mad'):
    if signal is not None:
        if method == 'mad':
            #assuming a normal distribution 
            #https://stackoverflow.com/a/40619633
            shift, scale = np.median(signal), np.float(statsmodels.robust.mad(signal))
        elif method == 'zscore':
            shift, scale = np.mean(signal), np.float(np.std(signal))
        else:
            raise ValueError('Normalized method not recogized')
            
        #signal normalization: https://nanoporetech.github.io/tombo/resquiggle.html#tombo-fast5-format
        # There may be problem after tombo 1.3, see explanation above
        norm_signal = (signal - shift) / scale #POTENTIAL ISSUE!
        assert len(signal) == len(norm_signal)
        norm_signal = np.around(norm_signal, 6)
        return norm_signal
    else:
        return None

# Write utilities

## write helper code

In [188]:
from ont_fast5_api.conversion_tools.conversion_utils import get_fast5_file_list
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from pyfaidx import Fasta
import multiprocessing

In [206]:
#xtract single fast5 file
fast5 = fast5_parser.raw_fast5(path, corr_group, basecall_group, basecall_subgroup, signal_group)

#Extract event information: #Raw signal --> Normalization
event = fast5.fast5_event()
raw_signal = fast5_rescale(path, fast5_signal)
norm_signal = fast5_normalized_signal(raw_signal,  method='mad')

basecalled_seq, raw_signal_list, norm_signal_list = "", [], []
for e in event:
#    print(e)
    basecalled_seq += e[2]
#    print(norm_signal[e[0]:(e[0] + e[1])])
    norm_signal_list.append(norm_signal[e[0]:(e[0] + e[1])]) #event start position: end position(start+length)
    raw_signal_list.append(raw_signal[e[0]:(e[0] + e[1])])
    assert len(norm_signal_list) == len(raw_signal_list)

# Extract other information
readid, fast5_signal = fast5.fast5_signal() #get readid
chrom, strand, start, read_strand = fast5.fast5_align()

In [207]:
basecalled_seq[0:10]

'ATTTCTGAAG'

### one hot coding

In [208]:
# https://2-bitbio.com/2018/06/one-hot-encode-dna-sequence-using.html
# https://gist.github.com/rachidelfermi/7fce95fd67e67fa47681e2f7d206c5a3
basepair = {'A': 'T',
            'C': 'G', 
            'G': 'C', 
            'T': 'A'}


class hot_dna:
    def __init__(self, seq):
        seq_array = np.array(list(seq))
        #integer encode the sequence
        seq_integer = LabelEncoder().fit_transform(seq_array) 

        #reshape because that's what OneHotEncoder likes
        seq_integer = seq_integer.reshape(len(seq_integer), 1)
        seq_1hot = OneHotEncoder(sparse=False).fit_transform(seq_integer)
        
        self._seq = seq_array
        self._integer = seq_integer
        self._1hot = seq_1hot
        
seq_1hot = hot_dna(basecalled_seq)._1hot
seq_1hot

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

### Get refererence length 
The same function as `get_contig2len` in DeepSignal, but more straightforward

In [159]:
def get_ref_length(ref_path):
    ref_length = dict()
    
    #case-insensitive, and to eliminate a potentially large else-if chain:
    while ref_path.lower().endswith(('.fasta', '.fa')) is True:
        try:
            for ref in Fasta(ref_path,"fasta"):
                ref_length[ref.name] = len(ref) #add ref_name:ref_length pair into dictionary
            return ref_length
        except TypeError:
            print("The reference must be .fa or .fasta format!")

ref_path = '/pod/2/li-lab/Ziwei/Nanopore/data/reference/T4_147.fa'
ref_length = get_ref_length(ref_path)

### Define put_file_in_queue
I borrow the code from `_fill_files_queue` from Deepsignal

In [173]:
def put_file_in_queue(fast5_q, fast5_file, batch_size):
    for i in np.arange(0, len(fast5_file), batch_size):
        fast5_q.put(fast5_file[i:(i+batch_size)])
    return

### Define process_fast5 function

I modified the code `_extract_preprocess` from Deepsignal

In [184]:
def preprocess_helper(fast5_path, ref_path, 
                      motif = 'CG', batch_size = 100):

    #batch_size: number of fast5 files to be processed by each process one time
    
    #Extract fast5 files in a list 
    fast5_files = get_fast5_file_list(fast5_path, recursive=True)
    print("Find {} fast5 files!".format(len(fast5_files)))

    print("Read genome reference..")
    ref_length = get_ref_length(ref_path)

    fast5_q = multiprocessing.Queue()
    put_file_in_queue(fast5_q, fast5_files, batch_size)
    
    print("Preprocess is done!")

    return motif, ref_length, fast5_q, len(fast5_files)

In [185]:
fast5_path = '/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
ref_path = '/pod/2/li-lab/Ziwei/Nanopore/data/reference/hg38.fa'

motif, ref_length, fast5_q, fast5_length = preprocess_helper(fast5_path, ref_path)

Find 3 fast5 files!
Read genome reference..
Preprocess is done!


# Extract features

In [192]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S, %D")
print("Current Time =", current_time)

motif, ref_length, fast5_q, fast5_length = preprocess_helper(fast5_path, ref_path)

feature = multiprocessing.Queue()
error = multiprocessing.Queue()
    
print('Getting features from nanopore reads...')
    

Current Time = 00:02:57, 10/13/21
Find 3 fast5 files!
Read genome reference..
Preprocess is done!
Getting features from nanopore reads...


In [136]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
fast5_fn='00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5'

signal_group='Raw/Reads'
corr_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl

# Open multiple fast5 files

In [10]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
from ont_fast5_api.conversion_tools.conversion_utils import get_fast5_file_list

fast5_list = get_fast5_file_list(input_path,recursive=True)

for fast5_file in fast5_list:
#    print(fast5_file)
    fast5 = fast5_parser.raw_fast5(fast5_file, corr_group, basecall_group, basecall_subgroup, signal_group)
    readid, fast5_signal = fast5.fast5_signal()
    event = fast5.fast5_event()
    chrom, strand, start, read_strand = fast5.fast5_align()
    
    raw_signal = fast5_rescale(fast5_file, fast5_signal)
    norm_signal = fast5_normalized_signal(raw_signal,  method='mad')
    
              

[0.661999 0.312264 0.187358 ... 1.311508 1.286527 1.13664 ]
File 00047d2f-1825-4151-a702-f01d559f22ab.fast5 didn't have event info

File 00047d2f-1825-4151-a702-f01d559f22ab.fast5 didn't have alignment info
[-0.343161 -0.283996 -0.141998 ...  1.514644  1.467311  3.064787]
[-0.196215 -0.220742  0.183952 ... -1.115974 -0.858442  4.010148]


In [1]:
motif='CG'
set(motif)

{'C', 'G'}

In [2]:
set(set(motif))

{'C', 'G'}